# Hands-on session: Machine Learning & Data Visualization
---
## 2 - Fitting ML models
---


How do I run this?

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sina-mansour/OHBM-Brainhack-2021)

Gitpod?

Binder?

---

## Train-Test splitting

---
## Estimators and cost functions


---
## Regression estimators

---
## Classification estimators


---
## Clustering estimators

---
## Dimensionality reduction estimators
